### Import packages

In [88]:
library(GWmodel)      ### GW models
library(dplyr)
library(sp)           ## Data management
library(car)          ## vif
library(spdep)        ## Spatial autocorrelation
library(RColorBrewer) ## Visualization
library(classInt)     ## Class intervals
library(raster)       ## spatial data
library(grid)         # plot
library(gridExtra)    # Multiple plot
library(ggplot2)      # Multiple plot
library(gtable)
library(GGally)       # 相關係數圖矩陣（scatter plot matrix）
library(maptools)
library(MASS)
library(tmap)

### Function

#### pdf_plot

In [89]:

pdf_plot <- function(x){
  g = ggplot()+
    geom_histogram(aes(x = x, y = ..density..), 
                   fill = '#557C55', alpha = 0.8)+
    geom_density(aes(x = x, y = ..density..), 
                 color = '#062C30', size = 1)+
    theme_bw()
  
  return(g)
}

### Load datas

In [90]:
getwd()
path = '..\\..\\Roaming-Dogs-Data\\'
Variable_KS_df <- read.csv(paste0(path, "@Test_KS\\Variable.csv" ), fileEncoding = 'utf-8')
Variable_KS_df["Clinic"][is.na(Variable_KS_df["Clinic"])] = 0
Variable_KS_shp<-shapefile(paste0(path, "@Test_KS\\Variable.shp" ),encoding = 'big5')
crs(Variable_KS_shp) <- CRS('+init=EPSG:3826') 
# Variable_KS_shp_02 = st_read(paste0(path, "@Test_KS\\Variable.shp"))
# Variable_KS_centroid <-  st_centroid(Variable_KS_shp)
# colnames(Variable_KS_centroid)

[1] "c:/@@Roaming_Dogs_Data_Analysis/Roaming-Dogs-Code/Model-fitting"

Warning message in OGRSpatialRef(dsn, layer, morphFromESRI = morphFromESRI, dumpSRS = dumpSRS, :
"Discarded datum Taiwan_Datum_1997 in Proj4 definition: +proj=tmerc +lat_0=0 +lon_0=121 +k=0.9999 +x_0=250000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
Warning message in showSRID(uprojargs, format = "PROJ", multiline = "NO", prefer_proj = prefer_proj):
"Discarded datum Taiwan_Datum_1997 in Proj4 definition"


### Create a residual dataframe

### select Xy

In [91]:
col_X = 
c( "Cluster", # 分群
"Market","Hospital" ,  "Temple",  "Ele" ,"Junior" ,"Senior", "Train.stat",  "Clinic", # 公共建設
"high_rat",   "mid_rat","low_rat", "M_F_RAT" , "P_DEN", "YOUN_DEP","OLD_DEP","AGING_IDX", # 人口統計(教育程度、人口密度...)
"Income_mea", "Income_med" ,"Income_Q1",  "Income_Q3",  "Income_sta") # 村里收入
col_y = c('Nt')

In [92]:
for(i in c('Market','Hospital',  "Temple",  "Ele" ,"Junior" ,"Senior", "Train.stat",  "Clinic")){
        
    i_new = paste0(i, "_den")
    print(i_new)
    
    Variable_KS_shp@data[i_new] = Variable_KS_shp@data[i]/Variable_KS_shp@data$Area_sqkm
    Variable_KS_df[i_new] = Variable_KS_df[i]/Variable_KS_df$Area_sqkm
}

[1] "Market_den"
[1] "Hospital_den"
[1] "Temple_den"
[1] "Ele_den"
[1] "Junior_den"
[1] "Senior_den"
[1] "Train.stat_den"
[1] "Clinic_den"


In [93]:
col_X = 
c( "Cluster", # 分群
"Market_den","Hospital_den" ,  "Temple_den",  "Ele_den" ,"Junior_den" ,"Senior_den", "Train.stat_den",  "Clinic_den", # 公共建設
"high_rat",   "mid_rat","low_rat", "M_F_RAT" , "P_DEN", "YOUN_DEP","OLD_DEP","AGING_IDX", # 人口統計(教育程度、人口密度...)
"Income_mea", "Income_med" ,"Income_Q1",  "Income_Q3",  "Income_sta")

In [94]:
for(i in c(col_X, col_y)){

    Variable_KS_shp@data[i] = sapply(Variable_KS_df[i], function(x) as.numeric(x))
}

#### Correlation 

In [77]:
corr = cor(Variable_KS_df[c(col_y,col_X)])
# col_income = c( "Income_mea","Income_med" ,"Income_Q1", 
#                 "Income_Q3",  "Income_sta", "Income_CV")
# corr_income = sort(corr[col_income,'Nt'])
idx = abs(corr[,'Nt'])>.1
col_X_02 = names(corr[idx,'Nt']) %>% tail(-1)

In [78]:
(corr[idx,'Nt']) %>% tail(-1) %>% sort
print(col_X_02)

P_DEN     high_rat   Income_mea   Income_sta      OLD_DEP    Income_Q3 
  -0.5689474   -0.3271878   -0.3221309   -0.3088155   -0.2962396   -0.2873985 
  Clinic_den   Income_med   Temple_den    Income_Q1      Ele_den     YOUN_DEP 
  -0.2813614   -0.2587846   -0.2479040   -0.2116341   -0.1904309   -0.1651909 
Hospital_den      mid_rat    AGING_IDX   Junior_den   Market_den      low_rat 
  -0.1412590   -0.1362949   -0.1112342   -0.1076914   -0.1044481    0.3520512 
     M_F_RAT      Cluster 
   0.3697244    0.3828319

 [1] "Cluster"      "Market_den"   "Hospital_den" "Temple_den"   "Ele_den"     
 [6] "Junior_den"   "Clinic_den"   "high_rat"     "mid_rat"      "low_rat"     
[11] "M_F_RAT"      "P_DEN"        "YOUN_DEP"     "OLD_DEP"      "AGING_IDX"   
[16] "Income_mea"   "Income_med"   "Income_Q1"    "Income_Q3"    "Income_sta"  


### GW PCA
-  [link01](https://gis.stackexchange.com/questions/35159/how-can-i-conduct-geographically-weighted-principal-component-analysis-using-arc)

In [95]:
DM<-gw.dist(dp.locat=data.matrix(((Variable_KS_df[c('X', "Y")]))))


bw.gwpca.basic <- 
bw.gwpca(Variable_KS_shp, vars = col_X_02, k =5, robust = FALSE, adaptive = TRUE, dMat = DM)
# bw.gwpca.robust <- 
# bw.gwpca(Variable_KS_shp, vars = col_X_02, k = 5, robust = TRUE, adaptive = TRUE, dMat = DM )

Warning message in proj4string(data):
"CRS object has comment, which is lost in output"


Adaptive bandwidth(number of nearest neighbours): 36 CV score: 1125214 
Adaptive bandwidth(number of nearest neighbours): 23 CV score: 1581478 
Adaptive bandwidth(number of nearest neighbours): 44 CV score: 875981.9 
Adaptive bandwidth(number of nearest neighbours): 49 CV score: 821830.9 
Adaptive bandwidth(number of nearest neighbours): 52 CV score: 909113.8 
Adaptive bandwidth(number of nearest neighbours): 46 CV score: 840732 
Adaptive bandwidth(number of nearest neighbours): 49 CV score: 821830.9 


In [96]:
bw.gwpca.basic

[1] 49

In [100]:
gwpca.basic <- gwpca(Variable_KS_shp,
 vars = col_X_02, bw = bw.gwpca.basic, k = 8, robust = FALSE, adaptive = TRUE)

Warning message in proj4string(data):
"CRS object has comment, which is lost in output"


In [102]:
prop.var <- function(gwpca.obj, n.components) {

    return((rowSums(gwpca.obj$var[, 1:n.components]) /rowSums(gwpca.obj$var)) * 100)
     }

In [105]:
prop.var(gwpca.basic, 3)

[1] 99.99455 99.99454 99.99447 99.99450 99.99476 99.99512 99.99512 99.99461
 [9] 99.99459 99.99460 99.99468 99.99458 99.99447 99.99451 99.99452 99.99548
[17] 99.99577 99.99607 99.99633 99.99527 99.99196 99.98994 99.98972 99.99483
[25] 99.99490 99.99554 99.99117 99.99198 99.99024 99.99080 99.99561 99.99526
[33] 99.99526 99.99519 99.99481 99.99449 99.99456 99.99460 99.99469 99.99456
[41] 99.99455 99.99453 99.99458 99.99462 99.99474 99.99474 99.99452 99.99569
[49] 99.99556 99.99465 99.99482 99.99450 99.99455 99.99472 99.99469 99.99481
[57] 99.99456 99.99455

### GLM_POISSON 

In [ ]:
Fit_Po <-glm(Nt~.,data=Variable_KS_df[,c(col_y,col_X_02)],family=poisson())
# Fit_Po <- glm(Nt~., data=Variable_KS_df[,c(col_y,col_X_02)], family = Gamma(link = "log"))
summary(Fit_Po) #查看回归模型参数


In [ ]:
# pdf_plot(Fit_Po$residuals)+xlab('residuals')
Variable_KS_centroid %>% 
  ggplot()+geom_sf(aes(color = residuals(Fit_Po), size = residuals(Fit_Po), alpha = .8))+
  scale_fill_gradient(low = "#56B1F7", high = "#132B43", na.value = NA)+
  theme_bw()

In [ ]:
nb <- poly2nb(Variable_KS_shp, queen=TRUE)
lw <- nb2listw(nb, style="W", zero.policy=TRUE)

In [ ]:
print(lw$weights[[1]])

### Spatial glmm
- [Package](https://kimura.univ-montp2.fr/~rousset/spaMM/spaMMintro.pdf)
- [Theory](https://bookdown.org/xiangyun/Thesis-Template-Bookdown/)

### GWR-Poisson

In [ ]:
formula.GWPR = Nt ~.
DM<-gw.dist(dp.locat=data.matrix(((Variable_KS_df[c('X', "Y")]))))

In [ ]:
Variable_KS_df$Nt = as.numeric(Variable_KS_df$Nt )
Variable_KS_shp$Nt = Variable_KS_df$Nt

#### GW PCA

In [ ]:
bw.gwpca(data,vars,k=2, robust=FALSE,kernel="bisquare",adaptive=FALSE,p=2, 
         theta=0, longlat=F,dMat)